### Use case notebook: schedule CCB team meetings ###
Date: 05/12/2025

In [2]:
import os
import glob
import numpy as np
from collections import deque

# There is a warning in the timeboard library that we want to suppress here
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
import time
import timeboard as tb
import timeboard.calendars.US as US
import datetime
import holidays

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the package
%load_ext autoreload
%autoreload 2
import cadence
from cadence.utils import FileOP
from cadence.mscheduler import Meetings
from cadence.mscheduler import cyclic_permutate
print(f'Package version: {cadence.__version__}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Package version: 0.0.0


### Meeting participants and settings ###

In [3]:
# Docker drive
data_dir = os.path.join(os.environ.get('HOME'), 'data')

group_members_file_name = 'ccb_members_2025.xlsx'
member_file = os.path.join(data_dir, group_members_file_name)

tm_raw = pd.read_excel(member_file)
display(tm_raw)

name                                         email group
0   Andreas               andreas_werdich@hms.harvard.edu    ai
1    Gerald               gerald_kiwanuka@hms.harvard.edu    ai
2    Nathan                 nathan_palmer@hms.harvard.edu    ai
3     Nidia        nidia_bucarellisanchez@hms.harvard.edu    ai
4      Alex                alex_pickering@hms.harvard.edu  cbio
5    Andrew                  andrew_ghazi@hms.harvard.edu  cbio
6   Anthony  anthony-alexander_christidis@hms.harvard.edu  cbio
7      Grey                   grey_kuling@hms.harvard.edu  cbio
8    Ludwig            ludwig_geistlinger@hms.harvard.edu  cbio
9      Tram                   tram_nguyen@hms.harvard.edu  cbio
10   Tyrone                    tyrone_lee@hms.harvard.edu  cbio

### Cleanup of the spreadsheet ###
Just in case that was not done. To provide a consistent look.

In [4]:
email_col = 'email'
name_col = 'name'
group_col = 'group'

# Lets do some cleanup of the spreadsheet
# small letters for the column names
tm = tm_raw.copy()
tm.columns = [col.lower() for col in tm.columns]

# small letters for emails and group, first letter capitalize for names
tm[email_col] = tm[email_col].str.lower()
tm[group_col] = tm[group_col].str.lower()
tm[name_col] = tm[name_col].str.title()

# Sort by group and then name
tm = tm.sort_values(by=['group', 'name'], ascending=True).reset_index(drop=True)

# Mark names that are not presenting
rm_name = ['nathan', 'ludwig']
rm_name = [nm.title() for nm in rm_name]
tm = tm.assign(presenting=True)
tm.loc[tm['name'].isin(rm_name), 'presenting'] = False

# save the name ist as a csv_file
presenter_list_file_name = f'{os.path.splitext(group_members_file_name)[0]}_list.csv'
presenter_list_file = os.path.join(data_dir, presenter_list_file_name)
tm.to_csv(presenter_list_file, index=False)
display(tm)

name                                         email group  presenting
0   Andreas               andreas_werdich@hms.harvard.edu    ai        True
1    Gerald               gerald_kiwanuka@hms.harvard.edu    ai        True
2    Nathan                 nathan_palmer@hms.harvard.edu    ai       False
3     Nidia        nidia_bucarellisanchez@hms.harvard.edu    ai        True
4      Alex                alex_pickering@hms.harvard.edu  cbio        True
5    Andrew                  andrew_ghazi@hms.harvard.edu  cbio        True
6   Anthony  anthony-alexander_christidis@hms.harvard.edu  cbio        True
7      Grey                   grey_kuling@hms.harvard.edu  cbio        True
8    Ludwig            ludwig_geistlinger@hms.harvard.edu  cbio       False
9      Tram                   tram_nguyen@hms.harvard.edu  cbio        True
10   Tyrone                    tyrone_lee@hms.harvard.edu  cbio        True

In [5]:
# Remove the names that we do not want in the schedule
presenters_df = tm.loc[tm['presenting']==True]
display(presenters_df)

name                                         email group  presenting
0   Andreas               andreas_werdich@hms.harvard.edu    ai        True
1    Gerald               gerald_kiwanuka@hms.harvard.edu    ai        True
3     Nidia        nidia_bucarellisanchez@hms.harvard.edu    ai        True
4      Alex                alex_pickering@hms.harvard.edu  cbio        True
5    Andrew                  andrew_ghazi@hms.harvard.edu  cbio        True
6   Anthony  anthony-alexander_christidis@hms.harvard.edu  cbio        True
7      Grey                   grey_kuling@hms.harvard.edu  cbio        True
9      Tram                   tram_nguyen@hms.harvard.edu  cbio        True
10   Tyrone                    tyrone_lee@hms.harvard.edu  cbio        True

### Shuffle the data and create a presenter list ###

In [6]:
# Let's do a random shuffling of the data frame
random_state = 234
presenters_df_shuffled = presenters_df.sample(frac=1, random_state=random_state)

# Get the names and the groups from the new data frame
name_list = list(presenters_df_shuffled['name'].values)
group_list = list(presenters_df_shuffled['group'].values)

# Instantiate the Meetings class with the list of names and groups from the data frame
meet = Meetings(name_list=name_list, group_list=group_list)

# We create a presenter list by merging the groups so that we have a member 
# from a different group presenting each time
name_sequence = meet.create_name_sequence()
# Rotate the sequence (start with a specific name)
rotated_sequence = cyclic_permutate(name_sequence, name='Grey')
updated_sequence = ['Grey', 'Andrew','Alex', 'Gerald', 'Tram', 'Andreas', 'Anthony', 'Nidia', 'Tyrone']
new_presenter_list = meet.create_name_sequence(name_sequence=updated_sequence, merge_groups=False)
display(meet.name_df)

name group
0     Grey  cbio
1   Andrew  cbio
2     Alex  cbio
3   Gerald    ai
4     Tram  cbio
5  Andreas    ai
6  Anthony  cbio
7    Nidia    ai
8   Tyrone  cbio

### Create the meeting schedule ###

In [9]:
start_date = '2025-04-16'
end_date = '2025-08-27'

cal = meet.create_timeboard(start_date=start_date, 
                            end_date=end_date,
                            start_name='Tyrone')
skip_date = '2025-04-16'
skip_name = 'CCB'
skip_comment = 'HMS All-Staff Town Hall'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

skip_date = '2025-05-21'
skip_name = 'Andrew'
skip_comment = 'Follow-up from 05/07/2025'

cal = meet.skip_date(cal_df=cal,
                     date=skip_date,
                     comment=skip_comment,
                     name=skip_name)

display(cal)

date     name group  holiday                    comment
0  2025-04-16      CCB  cbio    False    HMS All-Staff Town Hall
1  2025-04-23   Tyrone  cbio    False                       None
2  2025-04-30     Grey  cbio    False                       None
3  2025-05-07   Andrew  cbio    False                       None
4  2025-05-14     Alex  cbio    False                       None
5  2025-05-21   Andrew    ai    False  Follow-up from 05/07/2025
6  2025-05-28   Gerald    ai    False                       None
7  2025-06-04     Tram  cbio    False                       None
8  2025-06-11  Andreas    ai    False                       None
9  2025-06-18  Anthony  cbio    False                       None
10 2025-06-25    Nidia    ai    False                       None
11 2025-07-02   Tyrone  cbio    False                       None
12 2025-07-09     Grey  cbio    False                       None
13 2025-07-16   Andrew  cbio    False                       None
14 2025-07-23     Alex  cbio    False                       None
15 2025-07-30   Gerald    ai    False                       None
16 2025-08-06     Tram  cbio    False                       None
17 2025-08-13  Andreas    ai    False                       None
18 2025-08-20  Anthony  cbio    False                       None
19 2025-08-27    Nidia    ai    False                       None

In [10]:
# Save the presentation schedule
save_cols = ['date', 'name', 'comment']
schedule_name = f'ccb_presentations_2025_04.csv'
schedule_file = os.path.join(data_dir, schedule_name)
cal[save_cols].to_csv(schedule_file, index=False)
display(cal[save_cols])
print(schedule_file)

date     name                    comment
0  2025-04-16      CCB    HMS All-Staff Town Hall
1  2025-04-23   Tyrone                       None
2  2025-04-30     Grey                       None
3  2025-05-07   Andrew                       None
4  2025-05-14     Alex                       None
5  2025-05-21   Andrew  Follow-up from 05/07/2025
6  2025-05-28   Gerald                       None
7  2025-06-04     Tram                       None
8  2025-06-11  Andreas                       None
9  2025-06-18  Anthony                       None
10 2025-06-25    Nidia                       None
11 2025-07-02   Tyrone                       None
12 2025-07-09     Grey                       None
13 2025-07-16   Andrew                       None
14 2025-07-23     Alex                       None
15 2025-07-30   Gerald                       None
16 2025-08-06     Tram                       None
17 2025-08-13  Andreas                       None
18 2025-08-20  Anthony                       None
19 2025-08-27    Nidia                       None

/app/data/ccb_presentations_2025_04.csv
